学生証番号 :

名前 :

メールアドレス(法政大学) :

# B2-4 VGG16ネットワークを用いた学習

拡大したCIFAR10画像をVGGネットワークの構造で学習するモデルを構築する。

もともと用いていたImageNet用の1000次元の出力を取り外し、CIFAR10用の10次元の出力に取り換える。

<img src="https://drive.google.com/uc?export=view&id=1RlOO0sQDVt6Y8l9lF34yZHb5F7_IEZf8" width = 50%></img>

<img src="https://drive.google.com/uc?export=view&id=1G0WQ5MaVpHRCktqHBiGbHARPb_MlGIDd" width = 50%></img>


VGGネットワークは先ほどのB-1実験で試したネットワークよりかなり大規模である。
ここではCIFAR10データセットの学習を試みる。

ただし、深いネットワークに32$\times$32の小さい画像を入力すると、途中のpoolingで画像サイズがなくなってしまうため、ここの実験では（無駄だが）縦横約5倍の160$\times$160にして入力する。

In [ ]:
# [1-0]
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg16
from sklearn.model_selection import train_test_split

## データセットの準備

In [ ]:
# [1-1]
def get_data(data_name="CIFAR10"):
    if data_name == "CIFAR100":
        (X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
    else:
        (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
    y_train = y_train.flatten()
    y_test = y_test.flatten()
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# [1-2]
# 予測ラベルの名前を所持しておく
target_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]


def random_plot(X, y, predict=None):
    W = 10
    H = 5
    fig = plt.figure(figsize=(10, 20))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=0.3, hspace=0.10, wspace=0.10)
    for i in range(5):
        for j in range(10):
            x_tmp = X[y == j]
            idx = np.random.randint(len(x_tmp))
            x = x_tmp[idx].reshape(32, 32, 3)
            ax = fig.add_subplot(H, W, (i * 10) + j + 1, xticks=[], yticks=[])
            ax.imshow(x)
            if predict is not None:
                pred_tmp = predict[y == j]
                p = pred_tmp[idx]
                ax.set_title(f"{j} -> {p}")
            else:
                ax.set_title(target_names[j])

以下の `DATASET` に `CIFAR100` を指定すると、100クラスの分類タスクのCIFAR100のデータセットがダウンロードできる。
この実験は、CIFAR10を行うが、興味がある人は別途CIFAR100を試してみてもよい。

In [ ]:
# [1-3]
DATASET = "CIFAR10"
# CIFAR100を使用したい場合は以下をコメントアウト
# CIFAR100の場合はrandom_plotが全てのラベルを表示しないので注意
# DATASET = "CIFAR100"
(X_train, y_train), (X_test, y_test) = get_data(DATASET)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# [1-4]
# 学習用データを覗いてみる。
# 複数回実行すると画像も変化する。
random_plot(X_train, y_train)

### モデルの準備
B2-3の実験と違い、**pre-trainされていない**VGG16モデルをロードする。
入力サイズをVGG16のもともとの224$\times$224から、160$\times$160に変更する。

(注：これは、CIFAR10の画像がもともと32x32の大きさしかなく、VGG16にそのまま入力すると、途中のpoolingの処理で画像サイズが1より小さくなってしまうため、単純に縦横5倍にする処理のためである。＝本質的には大きな無駄だが、簡単な実行のため今回はこのようにしている）

include_top=Falseとして、Conv5_3をmax poolingしたところまでのネットワークになっており、プログラムではここまでのネットワークを"core"と呼んでいる。  

今回は、CIFAR10の画像の認識を行うので、新たにCIFAR10用に10次元の出力をつなげる.
（100種類の認識を行うCIFAR100なら100次元）

<img src="https://drive.google.com/uc?export=view&id=1RlOO0sQDVt6Y8l9lF34yZHb5F7_IEZf8" width = 50%></img>

<img src="https://drive.google.com/uc?export=view&id=1G0WQ5MaVpHRCktqHBiGbHARPb_MlGIDd" width = 50%></img>


<img src="https://drive.google.com/uc?export=view&id=17ielhT-JYIzLwmIENxcPLiO8O5n21YTc" width = 50%></img>



VGG16モデルの入力～Conv5_3後のmax poolingまでをdownload (include_top=Falseとすると、全結合層なしの部分のみダウンロード）する。  
weights=None としているので、事前学習の重みではなく、乱数で初期化された重みを利用。  
つまりここでは、このモデルの重みを乱数の状態から（ゼロから：スクラッチともいう）学習し、その様子と成果を観察する。

In [ ]:
# [1-5]
# VGG16をCIFAR10ように改良したモデルを読み込む関数を定義
def get_vgg16_model(data_name="CIFAR10"):
    num_output = 10 if data_name == "CIFAR10" else 100
    # change input image size 224x224 -> 160x160
    # conv5_3のあとのmax poolまでのネットワーク(core)の出力をflattenした後、
    # 全結合層を介して10次元の出力をつなげる
    model = keras.Sequential(
        [
            keras.Input(shape=(None, None, 3)),
            keras.layers.Lambda(
                lambda img: tf.image.resize(img, (160, 160)), name="resize"
            ),
            keras.layers.Lambda(
                tf.keras.applications.vgg16.preprocess_input, name="preprocess"
            ),
            # VGGのCNN部分
            keras.applications.VGG16(
                include_top=False,
                weights=None,
            ),
            keras.layers.Flatten(),
            keras.layers.Dense(num_output, activation="softmax"),
        ]
    )
    return model

In [ ]:
# [1-6]
# モデルの読みこみ
model = get_vgg16_model(DATASET)

# モデルの全体図の概要
model.summary()

In [ ]:
# [1-7]
# VGG部分の概要確認
model.layers[2].summary()

### モデルのセットアップ

In [ ]:
# [1-8]
# 誤差関数と最適化手法を設定する関数を作成する。
def model_setup(model, optimizer="adam", lr=0.001):
    if optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

In [ ]:
# [1-9]
model_setup(model)
# 改めてネットワークの構成を確認。モデルパラメータの数が学習パラメータの数と一致していること
# 学習されるパラメータの数を確認しておく。
# （これでも大規模な全結合層　flatten-4096-4096-1000 を外しているので総数は普通のVGG16 の1/10程度になっている）
model.summary()


# VGG16のスクラッチ学習
最初に学習前のモデルで予測を行い、性能を確認する。


In [ ]:
# [1-10]
# テストデータで評価する
loss_no_train, accuracy_no_train = model.evaluate(X_test, y_test, verbose=0)
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

学習にはかなりの時間がかかるため、実験時間内では2、3epoch程度のみの学習しか行わない。
学習の傾向に関しては事前に実行した時のログから確認する。

※ レポートには10epochで学習したモデルの結果のグラフや、予測などを確認してほしいので授業外でEPOCHの値を10にして全体を実行しなおすこと。すべて実行完了するまでに１時間ほどかかるが、ヘッダーの「ランタイム」タブの「再起動してすべてのセルを実行」を行うことで全体を実行してくれる。
１時間放置していると自動でセッションが切れてしまうので、定期的に確認すること。


In [ ]:
# [1-11]
# 実際に学習を行う。
EPOCH = 2
# ここを10にして再度実行する。
# EPOCH = 10
batch_size = 128

trainlog = model.fit(
    X_train, y_train, epochs=EPOCH, validation_split=0.1, batch_size=batch_size
)

（参考）以下が10epoch学習を行った時のログ。レポートでは必ず自分の実施データに基づいた結果の報告及び考察を行うこと。
```
Epoch 1/10
352/352 [==============================] - 584s 2s/step - loss: 2.6456 - accuracy: 0.3136 - val_loss: 1.4932 - val_accuracy: 0.4668
Epoch 2/10
352/352 [==============================] - 537s 2s/step - loss: 1.3005 - accuracy: 0.5424 - val_loss: 1.1119 - val_accuracy: 0.6076
Epoch 3/10
352/352 [==============================] - 537s 2s/step - loss: 0.9844 - accuracy: 0.6556 - val_loss: 0.8443 - val_accuracy: 0.7066
Epoch 4/10
352/352 [==============================] - 539s 2s/step - loss: 0.7829 - accuracy: 0.7253 - val_loss: 0.8082 - val_accuracy: 0.7214
Epoch 5/10
352/352 [==============================] - 539s 2s/step - loss: 0.6410 - accuracy: 0.7777 - val_loss: 0.7941 - val_accuracy: 0.7364
Epoch 6/10
352/352 [==============================] - 535s 2s/step - loss: 0.5138 - accuracy: 0.8228 - val_loss: 0.7513 - val_accuracy: 0.7556
Epoch 7/10
352/352 [==============================] - 533s 2s/step - loss: 0.3974 - accuracy: 0.8612 - val_loss: 0.8566 - val_accuracy: 0.7470
Epoch 8/10
352/352 [==============================] - 533s 2s/step - loss: 0.3035 - accuracy: 0.8943 - val_loss: 0.9897 - val_accuracy: 0.7284
Epoch 9/10
352/352 [==============================] - 533s 2s/step - loss: 0.2400 - accuracy: 0.9155 - val_loss: 1.0662 - val_accuracy: 0.7436
Epoch 10/10
352/352 [==============================] - 532s 2s/step - loss: 0.2007 - accuracy: 0.9306 - val_loss: 0.9919 - val_accuracy: 0.7514
```
学習が進んでいることがわかる。パラメターの数が多いため、過学習の傾向も確認できる。（どこからそれが言えるか？）

## 学習ログの可視化

In [ ]:
# [1-12]
# 学習曲線を描画する関数の作成
def plot_result(log):
    fig = plt.figure(figsize=(10, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(log.history["accuracy"])
    ax1.plot(log.history["val_accuracy"])
    ax1.set_title("Model accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"], loc="best")
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(log.history["loss"])
    ax2.plot(log.history["val_loss"])
    ax2.set_title("Model loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"], loc="best")
    plt.show()

In [ ]:
# [1-13]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog)

In [ ]:
# [1-14]
# 評価
loss_train, accuracy_train = model.evaluate(X_test, y_test, verbose=0)

# 学習前のスコアの表示
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

# 学習後のスコアの表示
print("===== 学習後 =====")
print(f"学習後の誤差 :  ", loss_train)
print(f"学習後の正解率 : ", accuracy_train)

## 課題 B2-4-1 モデルの学習時間と認識精度の関係およびエラー解析
VGG-16を学習するためにかかった時間はどのくらいかかったか。
また、学習したパラメータ数はいくつであったか。確認せよ。
また、それぞれの時の画像認識せ学習回数に対して、どの程度のどんな画像の認識に正解し、どんな画像の認識に失敗したか。以下のコードを参考に画像を確認して考察せよ。  
- 予測が正解しているデータの表示
```
# モデルの予測
test_predict = np.argmax(model.predict(X_test), 1)
# 正解しているデータの取得
true_idx = test_predict == y_test
# 正解しているデータの可視化
random_plot(X_test[true_idx], y_test[true_idx], test_predict[true_idx])
```

- 予測が間違っているデータの表示
```
# モデルの予測
test_predict = np.argmax(model.predict(X_test), 1)
# 間違えているデータの取得
false_idx = test_predict != y_test
# 間違っているデータの可視化
random_plot(X_test[false_idx], y_test[false_idx], test_predict[false_idx])
```

それぞれ実行毎にデータは変わっていくため、多く実行することで失敗の傾向が確認できる。

In [ ]:
# ここから実験を追加する。(自由にセルを増やしてOK)